In [13]:
import numpy as np
import pandas as pd
import re

import sys
from logparser import Drain

input_log_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\logs\HDFS_2k.log'


# data_list = []

# with open(input_file, 'r') as file:
#     for line in file:
#         columns = line.split()

#         if len(columns) >= 6:
#             data_list.append(columns[5:])

# file.close()

# for item in data_list:
#     print(item)


input_dir = "\logs"  # The input directory of log file
output_dir = "\parse"  # The output directory of parsing results
log_file_all = input_log_file # The input log file name
log_format = "<Date> <Time> <Pid> <Level> <Component>: <Content>"  # HDFS log format
# Regular expression list for optional preprocessing (default: [])
regex = [
    r"blk_(|-)[0-9]+",  # block id
    r"(/|)([0-9]+\.){3}[0-9]+(:[0-9]+|)(:|)",  # IP
    r"(?<=[^A-Za-z0-9])(\-?\+?\d+)(?=[^A-Za-z0-9])|[0-9]+$",  # Numbers
]
st = 0.5  # Similarity threshold
depth = 4  # Depth of all leaf nodes

# run on training dataset
parser = Drain.LogParser(
    log_format, indir=input_dir, outdir=output_dir, depth=depth, st=st, rex=regex
)
parser.parse(log_file_all)

structuredLog = open("HDFS_2k.log_structured.csv","r")
df = pd.read_csv(structuredLog)

print(df)

# selected_columns = df[['EventId', 'ParameterList']]

# # Combine 'EventId' and 'ParameterList' into a new column 'CombinedParameterList'
# selected_columns['CombinedParameterList'] = selected_columns.apply(lambda row: f"{row['EventId']}, {row['ParameterList']}", axis=1)

# # Save the result to a new CSV file
# selected_columns.to_csv('features.csv', index=False)





Parsing file: E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\logs\HDFS_2k.log
Total lines:  2000
Processed 50.0% of log lines.
Processed 100.0% of log lines.
Parsing done. [Time taken: 0:00:01.446507]
      LineId   Date    Time    Pid Level                     Component  \
0          1  81109  203615    148  INFO  dfs.DataNode$PacketResponder   
1          2  81109  203807    222  INFO  dfs.DataNode$PacketResponder   
2          3  81109  204005     35  INFO              dfs.FSNamesystem   
3          4  81109  204015    308  INFO  dfs.DataNode$PacketResponder   
4          5  81109  204106    329  INFO  dfs.DataNode$PacketResponder   
...      ...    ...     ...    ...   ...                           ...   
1995    1996  81111  101621  24902  INFO      dfs.DataNode$DataXceiver   
1996    1997  81111  101735  26595  INFO  dfs.DataNode$PacketResponder   
1997    1998  81111  101804  26494  INFO      dfs.DataNode$DataXceiver   
1998    1999  81111  101954  26414  INF

In [19]:
import csv

input_log_file = 'E:\Log_anomaly_detection\Log_anomaly_detection\sushanth\preprocess\logs\HDFS_2k.log_structured.csv'

def extract_data(input_file, output_file):
    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # Extract EventId and strings of the form "blk_*"
        data_list = [(row['EventId'], param) for row in reader for param in row['ParameterList'].strip("[]").replace("'", "").split(', ') if param.startswith('blk_')]

    # Write the extracted data to a new CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['EventId', 'BlockString']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for data in data_list:
            writer.writerow({'EventId': data[0], 'BlockString': data[1]})

# Usage example
extract_data(input_log_file, 'blk_eve_ids.csv')


In [31]:
import csv
from collections import defaultdict

def extract_and_group_data(input_file, output_file):
    block_events = defaultdict(list)

    with open(input_file, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        
        # Extract EventId and strings of the form "blk_*"
        for row in reader:
            event_id = row['EventId']
            for param in row['BlockString'].strip("[]").replace("'", "").split(', '):
                if param.startswith('blk_'):
                    block_id = param
                    block_events[block_id].append(event_id)

    # Write the grouped data to a new CSV file
    with open(output_file, 'w', newline='') as csvfile:
        fieldnames = ['BlockId', 'EventIds']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for block_id, event_ids in block_events.items():
            writer.writerow({'BlockId': block_id, 'EventIds': ', '.join(event_ids)})

# Usage example
extract_and_group_data('blk_eve_ids.csv', 'grouped_output_file.csv')
